In [1]:
import os 
from ipsl_dataset import IPSL_DCPP
import lightning.pytorch as pl
import torch
import hydra
import numpy as np
from ipsl_dataset import surface_variables
scratch = os.environ['SCRATCH']
work = os.environ['WORK']
run_name = ''
#6 month 
checkpoint_path = f'{work}/ipsl_dcpp/ipsl_dcpp_emulation/6l0y0nym/checkpoints/epoch=4-step=14125.ckpt'
# 2 year
checkpoint_path = f'{work}/ipsl_dcpp/ipsl_dcpp_emulation/0qo4adab/checkpoints/24_month_epoch=04.ckpt'
no_soil_checkpoint_5_year = f'{work}/ipsl_dcpp/ipsl_dcpp_emulation/kwsn8y3n/checkpoints/24_month_epoch=07.ckpt'
with_soil_checkpoint_5_year = f'{work}/ipsl_dcpp/ipsl_dcpp_emulation/jvlh5yoa/checkpoints/24_month_epoch=07.ckpt'

In [8]:
from hydra import compose, initialize
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")


In [9]:
checkpoint_no_soil = torch.load(no_soil_checkpoint_5_year,map_location=torch.device('cpu'))
checkpoint_with_soil = torch.load(with_soil_checkpoint_5_year,map_location=torch.device('cpu'))


In [17]:
test = IPSL_DCPP('test',60)
test_dataloader = torch.utils.data.DataLoader(test,batch_size=1,shuffle=False,num_workers=1)

In [18]:

soil_model = hydra.utils.instantiate(cfg.experiment.module,backbone=hydra.utils.instantiate(cfg.experiment.backbone,soil=True),dataset=test_dataloader.dataset)
soil_model.load_state_dict(checkpoint_with_soil['state_dict'])

no_soil_model = hydra.utils.instantiate(cfg.experiment.module,backbone=hydra.utils.instantiate(cfg.experiment.backbone,soil=False),dataset=test_dataloader.dataset)
no_soil_model.load_state_dict(checkpoint_no_soil['state_dict'])


<All keys matched successfully>

In [19]:
trainer = pl.Trainer(fast_dev_run=3)
output = trainer.predict(soil_model, test_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 3 batch(es). Logging and checkpointing is suppressed.
SLURM auto-requeueing enabled. Setting signal handlers.


Predicting DataLoader 0: 100%|██████████| 3/3 [00:10<00:00,  0.29it/s]


In [20]:
no_soil_output = trainer.predict(no_soil_model, test_dataloader)

SLURM auto-requeueing enabled. Setting signal handlers.


Predicting DataLoader 0: 100%|██████████| 3/3 [00:08<00:00,  0.36it/s]


In [52]:
surf_pred = output[0][4]['next_state_surface']
surf_batch = output[0][3]['next_state_surface']

In [53]:
land_mask = torch.tensor(np.load('data/land_mask.npy'))

In [54]:
masked_surf_pred = torch.where(land_mask ==1,surf_pred,torch.nan)
masked_surf_batch = torch.where(land_mask ==1,surf_batch,torch.nan)


In [58]:
masked_surf_pred.shape

torch.Size([1, 135, 143, 144])

In [60]:
(masked_surf_batch - masked_surf_pred).squeeze().nanmean(axis=(1,2))[var_index]

tensor(-1.6826e-10)

In [42]:
masked_surf_pred.squeeze().nanmean(axis=(1,2))[var_index]


tensor(1.9234e-08)

In [43]:
masked_surf_batch.squeeze().nanmean(axis=(1,2))[var_index]


tensor(1.9065e-08)

In [18]:
diffs = []
for i in range(3):
    diff = output[i][0] - no_soil_output[i][0]
    diffs.append(diff.squeeze())

In [ ]:
np.stack(diffs).mean(axis=0)

In [23]:
diff.squeeze()[var_index]

tensor(-0.0014)

In [29]:
x = torch.tensor(np.zeros((1,135)))

In [21]:
surface_variables[(7*13)-1]

'fDeforestToAtmos'

In [7]:
mean_acc_stacked = []
pred_next_stacked = []
batch_next_stacked = []
for mean_acc,pred_next,batch_next,cur in output:
    mean_acc_stacked.append(mean_acc)
    pred_next_stacked.append(pred_next)
    batch_next_stacked.append(batch_next)

In [36]:
var_index = surface_variables.index('gpp')

In [25]:
np.stack(pred_next_stacked).squeeze()[var_index]

276.80637

In [26]:
np.stack(batch_next_stacked).squeeze()[var_index]

277.16013

In [27]:
np.stack(mean_acc_stacked).squeeze()[var_index]

0.99321955

In [17]:
def get_time_series(output,var_index):
    pred_means = []
    batch_means = []
    for i in range(len(output)):
        pred = output[i][0]
        batch = output[i][1]
        pred_means.append(pred[:,var_index].flatten())
        batch_means.append(batch[:,var_index].flatten())
    pred_means = torch.concat(pred_means)
    batch_means = torch.concat(batch_means)
    return pred_means,batch_means

In [29]:
ps,bs = get_time_series(output,surface_variables.index(var_name))

In [31]:
gpp_climatology = climatology[:,surface_variables.index(var_name)]

In [44]:
def acc(predicted,modeled):
    accuracy = np.sqrt((predicted * modeled) / (np.power(predicted,2) * np.power(modeled,2)))
    return accuracy
    

In [45]:
acc(ps - gpp_climatology,bs - gpp_climatology)

tensor([0.6369, 0.5893, 1.0082, 1.1735, 0.8873, 1.3764, 0.9183, 0.6126, 0.5680,
        0.5719, 0.5765, 0.6221])

In [ ]:
import matplotlib.pyplot as plt
var_name = 'tas'

plt.plot(ps - gpp_climatology,label='pred')
plt.plot(bs - gpp_climatology,label='actual')
# plt.plot(climatology[:,surface_variables.index(var_name)],label='climatology')
plt.title(f'global mean of {var_name}')

plt.legend()
#DOCUMENT THESE RESULTS METHODICALLY 
plt.savefig(f'{work}/{var_name}_plot')

In [8]:
import xarray as xr
ds = xr.open_dataset(test.files[0])
shell = ds.isel(time=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for var in surface_variables:
    var_index = surface_variables.index(var)
    mae = np.abs(batch['next_state_surface'][0][var_index,:,:] - pred['next_state_surface'][0][var_index,:,:]).mean().item()
    print(var,mae)

In [13]:
var_index = surface_variables.index('nep')

In [14]:
import numpy as np

In [ ]:
shell['gpp'].data = np.abs(batch['next_state_surface'][0][var_index,:,:] - pred['next_state_surface'][0][var_index,:,:])
shell['gpp'].plot.pcolormesh()
plt.title('tas')
